#  Анализируем чеки

В этом задании мы будем работать с покупками и чеками. Смотреть за корреляциями в покупках довольно полезно.

> В 1992 году группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (Drug Store — формат разнокалиберных магазинов у дома). После анализа всех этих транзакций самым сильным правилом получилось «Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе». К сожалению, такое правило показалось руководству Osco Drug настолько контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали. Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с работы домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайший магазин. И мужья, не долго думая, совмещали приятное с полезным — покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения.

Для работы будем использовать датасет о продуктовых корзинах: https://www.kaggle.com/heeraldedhia/groceries-dataset

In [ ]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline

In [ ]:
!wget 'https://drive.google.com/u/0/uc?id=1SnaNzCXqNsNGtLTxLMw_MrureQN5NAy3' -O 'groceries.csv'

--2021-09-16 08:33:50--  https://drive.google.com/u/0/uc?id=1SnaNzCXqNsNGtLTxLMw_MrureQN5NAy3
Resolving drive.google.com (drive.google.com)... 172.217.214.101, 172.217.214.102, 172.217.214.139, ...
Connecting to drive.google.com (drive.google.com)|172.217.214.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mrmid5ro759tgrpa1pt1rqdfipbfjuov/1631781225000/09631068139126121875/*/1SnaNzCXqNsNGtLTxLMw_MrureQN5NAy3 [following]
--2021-09-16 08:33:51--  https://doc-00-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mrmid5ro759tgrpa1pt1rqdfipbfjuov/1631781225000/09631068139126121875/*/1SnaNzCXqNsNGtLTxLMw_MrureQN5NAy3
Resolving doc-00-4o-docs.googleusercontent.com (doc-00-4o-docs.googleusercontent.com)... 108.177.112.132, 2607:f8b0:4001:c12::84
Connecting to doc-00-4o-docs.googleusercontent.com (doc-00-4o-docs.googleusercontent.co

Подружаем данные и смотрим как они выглядят.

In [ ]:
df = pd.read_csv('groceries.csv', sep=',')
df.columns = ['id', 'fielddate', 'product']
print(df.shape)
df.head()

(38765, 3)


,id,fielddate,product
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


## 1. Корреляции

Для начала поработаем с корреляциями в данных. 

__а)__ Какой товар покупался чаще всего? Сохраните название этого товара в переменную `product_name`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

product_name = ...


In [ ]:
# проверка, что задание решено корректно
assert len(product_name) == 10

__б)__ Сколько всего уникальных заказов было сделано? Сохраните число заказов в переменную `n_cnt`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

n_cnt = ...


In [ ]:
# проверка, что задание решено корректно
assert n_cnt > 3800
assert n_cnt < 4000


В таблице выше в каждой строке записана информация о покупке конкретного товара. Давайте сделаем табличку размера "число товаров" на "число покупок", чтобы понимать какие товары покупались вместе, а какие нет. 

> Обратите внимание, то здесь задание немного упрощено. Вообще говоря, нам нужно делать агрегацию по паре `fielddate, id`, если мы хотим изучать чеки по-честному. Но мы делаем её только по `id` для того, чтобы не усложнять задание. В качестве необязательного дополнения вы можете после сдачи задания переделать код так, чтобы дата тоже учитывалась при расчётах. 

In [ ]:
sparse_sales = pd.pivot_table(df, 
               values='fielddate', 
               index='id', 
               columns='product', 
               fill_value=0, aggfunc='count')

sparse_sales.head()

В нашей матрице огромное число нулей. Обычно такие матрицы называют разряжеными. Мы занимаем нулями кучу свободной памяти, которую мы могли бы не занимать, если бы хранили данные [в ином виде.](https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/)

__в)__ Постройте матрицу корреляций Пирсона. Для этого используйте метод таблицы `.corr`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

sales_correlation = ...


Какие продукты сильнее всего коррелируют с яйцами, `domestic eggs` (их чаще всего покупают вместе)?  Сохраните название самого скоррелированного продукта в переменную `top_1`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

top_1 = ...


Какие продукты "мешают" купить яйца, то есть отрицательно коррелируют с их покупкой? Сохраните название продукта с самой большой отрицательной корреляцией в переменную `bottom_1`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

bottom_1 = ...


In [ ]:
# проверка, что задание решено корректно
assert len(bottom_1) == 8
assert len(top_1) == 12


Напишите код, который выводит самые коррелируемые товары для случайного продукта из списка `unique_products`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

unique_products = df['product'].unique()

# your code here


__г)__ Какие два продукта коррелируют сильнее всего? Положите их название в лист `answer`

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = ... 

# your code here


In [ ]:
# проверка, что задание решено корректно
assert 'soups' in answer

Конечно же, корреляция — это [не единственный способ искать](https://habr.com/ru/company/ods/blog/353502/) между покупками ассоциативные правила.

## 2. Зависимость. 

В лекции мы с вами сказали, что события $A$ и $B$ называются независимыми, если $P(AB) = P(A)\cdot P(B)$. Отталкиваясь от этого определения, можно ввести другую характеристику, которая показывает, насколько продукты зависят друг от друга, а именно __поддержку (lift).__ 

$$
lift = \frac{P(AB)}{P(A)\cdot P(B)}
$$

Эта метрика описывает отношение зависимости товаров к их независимости. Если оказалось, что `lift = 1`, это означает, что покупка товара $A$ не зависит от покупки товара $B$. Если `lift > 1`, то это означает, что вероятность встретить оба товара в чеке, $P(AB)$ высокая, то есть товары покупают вместе. Если `lift < 1`, это означает, что товары, наоборот, очень часто покупают по-отдельности. 

__д)__ Посчитайте значение нашей метрики для яиц и молока (`'whole milk', 'domestic eggs'`). Запишите получившиеся значение метрики в переменную `answer`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = ...

# your code here


In [ ]:
# проверка, что задание решено корректно
assert answer < 3
assert answer > 1


__е)__ Посчитайте значение метрики для всех пар продуктов из датасета. Сохраните значения в словарик `dict`. В качестве ключа используете кортеж из пары продуктов. Чтобы удобнее было перебрать все сочетания, используйте `combinations` из модуля `itertools`.

Чтобы при подсчётах не возникало деления на ноль, добавьте к знаменателю маленькое число, например `1e-10`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


In [ ]:
import itertools as it
comb = list(it.combinations(unique_products, 2))

In [ ]:
test = []
for i in range(len(comb)):
    t = len(sparse_sales[(sparse_sales[comb[i][0]]==True)&(sparse_sales[comb[i][1]]==True)])
    if (t == 0) and food_lift[comb[i]] > 0:
        print(comb[i])
        break
    if t != 0:
        test.append(comb[i])

In [ ]:
comb[i]

In [ ]:
sparse_sales[(sparse_sales[comb[i][0]]==True)&(sparse_sales[comb[i][1]]==True)]

In [ ]:
food_lift[('whole milk', 'cream')]

In [ ]:
(sparse_sales [['whole milk', 'cream']] >= 1).all(axis=1).sum()

In [ ]:
ab = (sparse_sales [['whole milk', 'cream']] >= 1).all(axis=1).mean()
a = (sparse_sales ['whole milk'] >= 1).mean()
b = (sparse_sales ['cream'] >= 1).mean()
answer = ab/(a*b)
answer

In [ ]:
len(set(test))

Сколько пар продуктов покупали вместе хотя бы раз? Запишите ответ в переменную `answer`.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

answer = ...

# your code here


Для какой пары продуктов метрика $lift$ оказалась самой большой? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Сколько раз эти продукты встретились в выборке? Как думаете адеватно ли делать выводы по такому объёму данных? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Для какой пары продуктов метрика оказывается самой маленькой? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


In [ ]:
# проверка, что задание решено корректно

assert answer < 10000
assert answer > 9000


## 3. Неоцениваемые задания

Выше мы увидели, что некоторые продукты встречаются в выборке очень редко. Понятное дело, что по ним у нас не получится построить хорошее ассоциативное правило. Попробуйте повторить расчёт той же метрики, но с условием что продукт покупали больше 10 раз. Изучите самые покупаемые вместе продукты и самые непокупаемые вместе продукты. Насколько сильно список отличается от полученного в предыдущем задании? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here


Иногда в чеках пытаются искать __продукты-якоря.__ То есть продукты, которые являются основными. Например: айфон - основной продукт, наушники и чехол - дополнения к нему. Подумайте как можно попытаться найти такие продукты на основе простых метрик, основанных на подсчёте условных вероятностей.

<center>
<img src="https://pp.userapi.com/c638028/v638028181/52e5e/1X-dkzNN1hk.jpg" width="400"> 
</center>